In [1]:
import os
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/research'

In [2]:
os.chdir("../")
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers'

In [3]:
# creating entity
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelTrainerConfig:
   root_dir: Path
   data_path: Path
   tokenizer_name: Path
   num_train_epochs: int
   warmup_steps: int
   per_device_train_batch_size: int
   per_device_eval_batch_size: int
   weight_decay: float
   logging_steps: int
   evaluation_strategy: str
   eval_steps: int
   save_steps: int
   gradient_accumulation_steps: int

In [4]:
# building the configuration manager
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
            num_train_epochs =  params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [ ]:
# build components

# internal imports
from src.textSummarizer.logging import logger

# external imports 
import os
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cuda"
        
        data_file_path = self.config.data_path
        dataset_samsum_pt = load_from_disk(data_file_path)

        model_name = self.config.tokenizer_name
        trainer_args = TrainingArguments(
                            output_dir=self.config.root_dir,
                            num_train_epochs= self.config.num_train_epochs,
                            warmup_steps= self.config.warmup_steps,
                            per_device_train_batch_size= self.config.per_device_train_batch_size,
                            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
                            weight_decay= self.config.weight_decay,
                            logging_steps= self.config.logging_steps,
                            do_eval=True,
                            eval_steps= self.config.eval_steps,
                            save_steps= self.config.save_steps,
                            gradient_accumulation_steps= self.config.gradient_accumulation_steps,
                            no_cuda = True, # forcing the usage of CPU
                            )
        tokenizer = AutoTokenizer.from_pretrained(model_name)  # load tokenizer
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # load model cnn_model
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        trainer = Trainer(model = model_pegasus,
                  args = trainer_args,
                  tokenizer = tokenizer,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["test"],      # since the train dataset is large in size
                  eval_dataset = dataset_samsum_pt["validation"]
                  )
        
        trainer.train()

        #save trained model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer_comp = ModelTrainer(model_trainer_config)
model_trainer_comp.train()

[2026-01-23 06:03:08 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts]
[2026-01-23 06:03:08 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts/model_trainer]


/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/.venv/lib/python3.11/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
